# 1: Filtrado y preprocesamiento
1. Cargue la información del archivo large_even.json en una lista, muestre la cantidad de registros total (deben ser 746, 909). Este es nuestro tráfico inicial!

In [34]:
import json
with open('large_eve.json', 'r') as f:
	data = [json.loads(line) for line in f]
print(f"Total de registros: {len(data)}")
print(data[:5])

Total de registros: 746909
[{'timestamp': '2017-07-22T17:33:15.784100-0500', 'flow_id': 283570730437312, 'pcap_cnt': 18519, 'event_type': 'alert', 'vlan': 130, 'src_ip': '192.168.203.200', 'src_port': 2328, 'dest_ip': '192.181.145.109', 'dest_port': 445, 'proto': 'TCP', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 2001569, 'rev': 15, 'signature': 'ET SCAN Behavioral Unusual Port 445 traffic Potential Scan or Infection', 'category': 'Misc activity', 'severity': 3}}, {'timestamp': '2017-07-22T17:33:16.571273-0500', 'flow_id': 1519048138171115, 'pcap_cnt': 21531, 'event_type': 'ssh', 'vlan': 140, 'src_ip': '192.168.204.60', 'src_port': 62990, 'dest_ip': '192.168.26.254', 'dest_port': 22, 'proto': 'TCP', 'ssh': {'client': {'proto_version': '2.0', 'software_version': 'OpenSSH_5.0'}, 'server': {'proto_version': '1.99', 'software_version': 'Cisco-1.25'}}}, {'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 

2. Debido a que estamos buscando dominios web, del total de registros, solamente estamos interesados en los registros DNS. Cargue únicamente aquellos registros que sean DNS.
3. Muestre la nueva cantidad de registros filtrados. Deben ser 21484. Esta es una cantidad mucho más manejable, pero aún se debe seguir depurando la información a buscar.

In [35]:
dns_filter = [json for json in data if json.get('event_type').lower() == 'dns']
print(f"Total de registros filtrando por DNS: {len(dns_filter)}")

Total de registros filtrando por DNS: 15749


4. Muestre la información de 2 registros cualesquiera.

In [15]:
print(dns_filter[0])
print(dns_filter[100])

{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}
{'timestamp': '2017-07-22T17:33:27.398815-0500', 'flow_id': 2090016796251365, 'pcap_cnt': 54532, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.207.4', 'src_port': 53, 'dest_ip': '192.168.205.170', 'dest_port': 36690, 'proto': 'UDP', 'dns': {'type': 'answer', 'id': 26728, 'rcode': 'NXDOMAIN', 'rrname': '<root>', 'rrtype': 'SOA', 'ttl': 20864}}


5. Debido a que la data consiste en estructuras JSON anidadas, utilice la característica json_normalize para normalizar la información y asignarla en un dataframe. Muestre el shape del dataframe, debería obtener (21484, 163).

In [36]:
import pandas as pd
from pandas import json_normalize

df_dns = json_normalize(dns_filter)
print(f"Shape: {df_dns.shape}")
df_dns.head()

Shape: (15749, 18)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


6. Como estamos buscando dominios DGA, debemos filtrar los registros DNS para aquellos registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio). Después de filtrar debería obtener 2849 registros.

In [ ]:
dns_type_a = df_dns[df_dns['dns.rrtype'] == 'A']
print(f"Registros DNS tipo A: {dns_type_a.shape[0]}")

Registros DNS tipo A: 2849


7. Filtre los dominios únicos. Debe obtener 177 registros únicos

In [ ]:
unique_domains = dns_type_a['dns.rrname'].unique()
print(f"Dominios únicos: {len(unique_domains)}")

Dominios únicos: 177


8. Escriba una función que obtenga el TLD para un dominio. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home. Utilice un LLM para ayudarle a escribir esta función, verifique que obtiene correctamente el TLD, incluya el prompt utilizado en su notebook.

Prompt:

En python provee una función que devuelva el TLD de un dominio; eg. Por ejemplo, para api.wunderground.com el TLD es wunderground.com, para safebrowsing.clients.google.com.home, el TLD es home.

In [19]:
import tldextract
import pandas as pd

unique_domains_df = pd.DataFrame(unique_domains, columns=['dns.rrname'])

def obtener_tld_para_dominio(dataframe, columna_dominio):
	dataframe['domain_tld'] = dataframe[columna_dominio].apply(lambda x: tldextract.extract(x).domain)
	return dataframe

df_a = obtener_tld_para_dominio(unique_domains_df, 'dns.rrname')
df_a.head()

,dns.rrname,domain_tld
0,api.wunderground.com,wunderground
1,stork79.dropbox.com,dropbox
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,aoltw
3,safebrowsing.clients.google.com.home,home
4,fxfeeds.mozilla.com,mozilla


9. Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando la función anterior para crear una columna nueva llamada domain_tld, y elimine todas las demás columnas.

In [20]:
df_a = df_a[['domain_tld']].copy()
df_a = df_a.drop_duplicates()
df_a.head()

,domain_tld
0,wunderground
1,dropbox
2,aoltw
3,home
4,mozilla


# 2: Data Science

10. Utilice Gemini para clasificar los dominios como DGA (1) o legítimos (0).

In [ ]:
import google.generativeai as genai
from tqdm import tqdm
import time

GEMINI_API_KEY = open("Gemini.key").read()
genai.configure(api_key=GEMINI_API_KEY)
modelo = genai.GenerativeModel('models/gemini-1.5-flash-001')

dominios_unicos = df_a['domain_tld'].tolist()

def clasificar_dominio(dominio):
	prompt = f"""
Los DGA suelen tener nombres aleatorios sin sentido; con combinaciones de letras y números, palabras de 2 o 3 letras sin sentido, palabras como malware o hacker o direcciones IP.
Los dominios legítimos tienen nombres reconocibles, con palabras reales o marcas conocidas.
Ejemplos de dominios como DGA (1) o legítimo (0):
- amazon.com (0)
- google.com (0)
- 450315uikl.com (1)
- 60wergndtggrd.net (1)
- com.test, aoltw, gogglesploit, melinga, me, informaction (1)

Clasifica el siguiente dominio basado en estos parametros: {dominio}
Responde únicamente con 0 o 1."""

	try:
		respuesta = modelo.generate_content(prompt)
		if "1" in respuesta.text.strip(): return 1
		elif "0" in respuesta.text.strip(): return 0
		else: return "NONE"
	except Exception as e:
		return f"Error: {e}"

resultados = {}
for dominio in tqdm(dominios_unicos, desc="Clasificando dominios"):
	clasificacion = clasificar_dominio(dominio)
	resultados[dominio] = clasificacion
	tqdm.write(f"{dominio} : {clasificacion}")
	time.sleep(5)

df_resultados = pd.DataFrame(list(resultados.items()), columns=['domain_tld', 'classification'])
df_resultados.to_csv("dominios.csv", index=False)
df_resultados.head()


#### Paso 11
11. Filtre aquellos considerados como DGA (valor 1 ) y muéstrelos.

In [ ]:
df_resultados = pd.read_csv("dominios.csv")
df_dga = df_resultados[df_resultados['classification'] == 1]
print(f"DGA: {len(df_dga)}")
df_dga.head()

DGA: 26


,domain_tld,classification
2,aoltw,1
5,metasploit,1
9,110phpmyadmin,1
10,informaction,1
14,ntkrnlpa,1


# 3: Dominio experto
#### Paso 12
Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo que debemos aplicar dominio experto para encontrar los verdaderos registros maliciosos. Escriba una función que utilice la lista de un millón de TLD proporcionada en Canvas, y devuelva 0 si el TLD se encuentra en la lista y 1 si no está. Utilice un LLM para crear dicha función, verifique que no se carga la lista cada vez que se busca un TLD. Incluya el prompt en su notebook.

Prompt utilizado:

Dado un archivo que contiene TLDs en formato id, domain top-1m.csv y un dataframe para verificar, devuelve 1 si es verdadero o 0 si no lo encuentra

In [ ]:
with open('top-1m.csv', 'r') as f:
	tld_list = [line.strip().split(',')[1] for line in f]

def check_tld_in_list(tld):
	return 0 if tld in tld_list else 1

df_dga['not_in_list'] = df_dga['domain_tld'].apply(check_tld_in_list)
df_suspicious = df_dga[df_dga['not_in_list'] == 1]

print(f"Sospechosos despues de filtrar: {df_suspicious.shape[0]}")

Dominios sospechosos después de filtrar por lista de TLDs: 26


C:\Users\50230\AppData\Local\Temp\ipykernel_45232\1597192605.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dga['not_in_list'] = df_dga['domain_tld'].apply(check_tld_in_list)


13. Utilice la función para determinar si los TLD se encuentran en dicha lista. Filtre aquellos que si se encuentran. Después de eliminar duplicados, debería obtener 13 dominios.

In [32]:
df_suspicious.head()

,domain_tld,classification,not_in_list
2,aoltw,1,1
5,metasploit,1,1
9,110phpmyadmin,1,1
10,informaction,1,1
14,ntkrnlpa,1,1


14. Finalmente, para confirmar los dominios maliciosos podemos buscar la fecha de creación del TLD. Cree una función qué en base al TLD, devuelva la fecha de creación de este. Utilice un LLM para escribir dicha función, incluya el prompt utilizado en su notebook.

Prompt Utilizado:

Cree una función qué en base al TLD, devuelva la fecha de creación de este.

In [33]:
import whois

def get_date(tld):
	try:
		tld = tld + '.com'
		w = whois.whois(tld)
		if w.creation_date:
			return w.creation_date
		else:
			return "NA"
	except Exception as e:
		return "NA"

def apply_date(df):
	df['creation_date'] = df['domain_tld'].apply(get_date)
	return df

df_con_fechas = apply_date(df_suspicious)
df_con_fechas.head()

,domain_tld,classification,not_in_list,creation_date
2,aoltw,1,1,NA
5,metasploit,1,1,NA
9,110phpmyadmin,1,1,NA
10,informaction,1,1,NA
14,ntkrnlpa,1,1,NA



15. Muestre la fecha de creación para cada uno de los 13 dominios finales ¿Cuáles son los dominios que podemos confirmar como sospechosos?

**No hubo datos para los dominios**

16. Recuerde que los dominios DGA son conocidos por formarse de forma aleatoria: secuencias aleatorias de caracteres, no palabras. Indique que dominios sospechosos tienen este patrón y que pueden confirmarse como dominios DGA.

**La mayoría de DGAs son letras aleatorias como: ntkrnlpa. "56""" ejfodfmfxlkgifuf**
